# Spectrum Sensing (Sparsity)

## Looking Ahead

In this series of examples, we are going to explore the use of compressive sensing for spectrum sensing in a simulated frequency-multiplexed channel sharing scheme. This first notebook will facilitate our exploration of the spectral content of simulated data from such a scheme, and expose the underlying sparsity of the frequency spectrum, which will make compressive sensing techniques applicable in later notebooks.

The precise nature of the model frequency-multiplexed transmission scheme is described in the homework. Here, we will load the simulated data from a file, remind ourselves of some points germane to working with RF signals, and then finally examine the frequency sparsity of the data.

## Downloading the Data

We download the data file from the Github page. The file is `ss_signal.mat`.

In [ ]:
!git clone https://github.com/hwagyesa/ss_data.git

## Loading and Understanding the Data

Next, we load the signal and the provided parameters into memory, and look at the available keys.

In [ ]:
import scipy.io as sio
import numpy as np
import bokeh.plotting as bpl
import pprint

D = sio.loadmat('/content/ss_data/ss_signal.mat', squeeze_me=True)
D_short = {k: v for k, v in D.items() if not k.startswith('_')}
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(D_short)

Here is a description of the contained parameters:
- `x`: The frequency-multiplexed channel's voltage signal. All subsequent parameters relate to different aspects of this signal.
- `SNR`: The signal-to-noise ratio of `x`, measured in decibels. Here we again emphasize that `x` is a voltage signal, and that we have added i.i.d. Gaussian random variables to each sample bin to ensure the specified SNR is met.
- `fmin`: The minimum frequency in the channel's allocated frequency band, measured in Hz.
- `B`: The bandwidth of a single user's subband allocation (in frequency), measured in Hz. 
- `Nuser`: The number of users, or subbands, in the allocated frequency range. We have the relation `fmax - fmin = Nuser * B`, so we can derive the maximum band frequency `fmax` from the previous three parameters.
- `fs`: The sampling frequency of the signal `x`, in Hz. Here we have used Nyquist-rate sampling, so `fs = 2 * fmax`.
- `Nslots`: This is a time-transmission parameter, which tells us how many subsequent transmission slots the signal contains.
- `slot_time`: Another time parameter that tells us the duration of a single transmission slot, measured in seconds. The total number of samples in `x` is thus equal to `fs * slot_time * Nslots`, and the total time duration of the signal is `slot_time * Nslots`.

Interpreting given the values above, we see that (for example) the slot bandwidth of our system is 100 kHz and the slot time is 1 ms, among other things.

### Derived Parameters

We compute some of the derived parameters mentioned above in the code block below.


In [ ]:
## Move dictionary vars into locals
B = D['B']
Nslots = D['Nslots']
Nuser = D['Nuser']
SNR = D['SNR']
fmin = D['fmin']
fs = D['fs']
slot_time = D['slot_time']
x = D['x']

## Derived parameters
fmax = Nuser * B + fmin
T = slot_time * Nslots
Nsamp = len(x)
slot_dur = Nsamp // Nslots
print(fmax)
print(fmin)

## Visualizing the Data

Now that we have specified the meaning of the different signal parameters, we can plot the frequency content of different transmission intervals of `x`. First, we generate frequency and time vectors which have the proper physical interpretation in light of our parameter specifications above.

In particular, our signal is sampled at the Nyquist rate, and so its (unrealizable) DTFT is `fs`-periodic, with no aliasing. This tells us what units to place on the frequency axis after we FFT the signal `x`. The time axis is similarly straightforward to determine.



In [ ]:
## Time and frequency axes
f_axis = fs * np.arange(-Nsamp/2+1, Nsamp/2+1)/Nsamp
t_axis = Nslots * slot_time * np.arange(0, Nsamp)/Nsamp
f_axis = np.squeeze(f_axis)
t_axis = np.squeeze(t_axis)

In [ ]:
## Plot the time signal, over a single slot time
bpl.output_notebook()
h = bpl.figure(title="Time-domain signal x")
h.xaxis.axis_label = 'Time (ms)'
h.yaxis.axis_label = 'Voltage (V)'
h.line(t_axis[0:slot_dur]*1.0e3, x[0:slot_dur])
bpl.show(h)

This plot does not tell us much about the characteristics of the signal `x`, which is of course more naturally viewed in the frequency domain. We plot this next.

**Note**: We use some tricks with the Fourier transform below, which you should make sure you understand. *First*, we use `fftshift` after calling `fft`: this is because the returned signal from `fft` represents the frequency content in the interval `[0, fs]`, whereas we are more used to having a DC-centered view of the frequency content, from `[-fs/2, fs/2]`. `fftshift` performs this adjustment by shuffling the array. *Second*, we also downsample the `f_axis` variable: this is because when we look at the frequency content in the first transmission slot, we have a shorter time duration to work with, so although the overall frequency band that we view is the same (we did not change the sampling frequency!), the frequency resolution becomes worse.

In [ ]:
slot1_x = x[0:slot_dur]
slot1_X = np.fft.fftshift(np.fft.fft(slot1_x))
## We need to downsample the frequency axis to match the shorter slot time
slot_f_axis = f_axis[::Nslots]

## Make the plot
bpl.output_notebook()
h = bpl.figure(title="Frequency-domain signal x, during first slot")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(slot_f_axis * 1e-6, 20 * np.log10(np.absolute(slot1_X)))
bpl.show(h)
print(slot1_X.shape)

We see that the signal consists of just a few different transmitters during the first transmission slot. We can get a closer view by plotting just our frequency band of interest, over the positive frequency axis.

In [ ]:
inband_mask = np.where((slot_f_axis >= fmin) & (slot_f_axis <= fmax))[0]

bpl.output_notebook()
h = bpl.figure(title="Frequency-domain signal x, during first slot (positive band-of-interest)")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(slot_f_axis[inband_mask] * 1e-6,
       20 * np.log10(np.absolute(slot1_X[inband_mask])))
bpl.show(h)

## Your Tasks

Each level three header below contains a task you should complete. See the homework handout for additional details.

### Task 1: Viewing Frequency Domain Signal In a Particular Subband

Now let's get an idea of which users are transmitting by zooming into a single subband, given our knowledge of the transmit scheme parameters. We know that the bands have bandwidth `B`, and that the frequency interval `[fmin, fmax]` is divided up into `Nuser` of them. From here we can compute the center frequency for each of the `Nuser` bands in the interval, and view their content by looking `B/2` above and below. 

1. Find out the center frequency of the 216th subband, and plot the frequency domain signal during the first slot of this subband. This should be a band with width `B`. 

2. Now plot a band with the same center, but with width `8B` (`[center - 4B, center + 4B]`). 

Your **output** should be two figures, showing the power of the frequency domain signal of the band we are interested. 

In [ ]:
frequencies=slot_f_axis[inband_mask]
#Since BW is 10k and every 1k frequency is sampled each band will get only 100 freq values.
#frequencies in each band = slot_f_axis[inband_mask].shape[0]/Nusers
band_frequencies=np.array(frequencies.reshape([256,100]))
band_values = np.array(slot1_X[inband_mask].reshape([256,100]))
#Here each row is a band and each band has 100 frequencies
subband_216=band_frequencies[215,:]
centerfreq_216=(subband_216[49]+subband_216[49])/2#Since the mid-point technically lies between 50th and 51st frequency for a 100 value interval

print(f'The center frequency for the 216th sub-band is {centerfreq_216* 1e-6} MHz')
#Plot 1
bpl.output_notebook()
h = bpl.figure(title="216th sub band frequency representation, during first slot (positive band-of-interest)")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(subband_216 * 1e-6,
       20 * np.log10(np.absolute(band_values[215,:])))
bpl.show(h)


#plot 2
#For this we would require the values from 213th to 219th sub bands completely(7B) and then later half of 212th and first half of 220th sub band(0.5B each)
BW_8 = np.concatenate((band_frequencies[211,50:],band_frequencies[212,:],band_frequencies[213,:],band_frequencies[214,:],band_frequencies[215,:],band_frequencies[216,:],
        band_frequencies[217,:],band_frequencies[218,:],band_frequencies[219,0:49]),axis=None)
X_BW_8=np.concatenate((band_values[211,50:],band_values[212,:],band_values[213,:],band_values[214,:],band_values[215,:],band_values[216,:],
        band_values[217,:],band_values[218,:],band_values[219,0:49]),axis=None)
bpl.output_notebook()
h = bpl.figure(title="8 BW from 216th sub band  representation, during first slot (positive band-of-interest)")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(BW_8 * 1e-6,
       20 * np.log10(np.absolute(X_BW_8)))
bpl.show(h)


### Task 2: Evaluating Subband Occupancy Over Time

The goal of this task is to generate a discrete analog of the time signal `x` that allows us to assess the subband-level sparsity of `x` during each time slot. Perform the following tasks:
1. Given the frequency content of a single time slot of `x`, write code to output a vector with `Nuser` entries: the `i`-th entry is 1 if the corresponding subband is occupied during this time slot and 0 otherwise. For example, you can expect as input the fft of a single time slot as we generated for the first timeslot in one of the cells above.
2. Apply the code you wrote in the previous step to estimate the length-`Nuser` vector of subband occupancies for each of the `Nslots` time slots contained in the signal `x`.
3. Visualize the matrix you get to intuitively understand how does users occupy subbands over time. 
4. Plot the density of the signal `x` as a function of the time slot? Here, density means "number of users transmitting simultaneously in a single time slot, divided by total number of users", and can be related to the sparsity of your zero-one matrix from the previous task. How do you interpret this density? 

For this part of the problem, your **output** should be the `Nuser`-by-`Nslots` matrix of subband occupancies for the signal `x`, an image that visualize your matrix, and the figure of density over time. 

**Hint**: You can use the fact that the SNR of the signal `x` is quite high to easily test whether a given subband is occupied or not: index it properly as we have done above (after computing the center frequencies of each subband), and apply something like a simple integrate-and-threshold test.


In [ ]:
threshold = 30
def each_slot_occupancy(X_slot,j):
  for i in range(Nuser):
      fmin = i*B
      fmax= fmin+B
      inband_mask = np.where((slot_f_axis >= fmin) & (slot_f_axis <= fmax))[0]
      masked_X = X_slot[inband_mask]
      power=20*np.log10(np.absolute(masked_X))
      
      if power.sum() > threshold:
        occupancy[i][j]=1
    



In [ ]:
def all_slot_occupancy(X):
  for i in range(Nslots):
    X_slot = X[i*slot_dur:i*slot_dur+slot_dur]
    X_slot = np.fft.fftshift(np.fft.fft(X_slot))
    each_slot_occupancy(X_slot,i)



In [ ]:
occupancy = np.zeros((256,16))
all_slot_occupancy(x)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.imshow(occupancy.astype(int),aspect='auto',interpolation ='nearest',cmap='hot')
plt.title("2D image showing the occupancy of each band for various time slots")
plt.xlabel("time slots")
plt.ylabel('freq bands in Mhz')

In [ ]:
density = np.sum(occupancy,axis=0)/256*100
plt.figure(figsize=(10,10))
plt.plot(np.arange(1,17),density)
plt.title("Density of occupancy for each time slot")
plt.xlabel("Time slot")
plt.ylabel("Density in percent")

As we see that throught time the density of the users that are actively using the sub bands are really less, hence at each time we expect a sparse vector with a lot of zeros and a few ones. To support this we see that the maximum density percent of the channel is just shy of 8% for the given time interval and usually is seen around the 5-6% occupancy mark. 